In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from geopy.distance import geodesic

In [2]:
df = pd.read_csv("refined_df.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

,Shipments: Attributes Ocean type category,Shipments: Attributes Shipment ID,Shipments: Attributes Tradelane,Shipments: Attributes Gross Weight (kg),Shipments: Attributes Volume (cbm),Shipment Containers: Distance Final Port to Destination Distance in mile,Shipment Containers: Distance Origin to First Port Distance in km,Shipments: Attributes Freight Type,Shipments: Locations Arrival port Coordinates,Shipments: Locations Departure port Coordinates,Port pair,Total Actual Days
0,coloaded_lcl,3051512,TAWB,323,2.622,160.13,358.66,Port to Door,"32.7764749,-79.93105120000001","53.5510846,9.9936819",DEHAM - USATL,70.0
1,coloaded_lcl,3065451,TAWB,490,0.538,30.83,346.84,Door to Door,"45.5016889,-73.567256","53.5510846,9.9936819",DEHAM - USCHI,45.0
2,coloaded_lcl,3022941,TPEB,"1,913",11.360,54.37,233.94,Port to Door,"34.0522342,-118.2436849","22.356851,91.7831819",BDCGP - USLAX,57.0
3,coloaded_lcl,3024506,FEWB,752,0.508,147.02,421.03,Door to Door,"51.961726,1.351255","18.9499361,72.9511875",INNSA - GBFXT,57.0
4,flxt_lcl,3055002,TPEB,"6,471",8.110,"1,793.56",296.68,Port to Door,"34.0522342,-118.2436849","22.6272784,120.3014353",TWKHH - USLAX,43.0


In [3]:
df[['dest_latitude', 'dest_longitude']] = df['Shipments: Locations Arrival port Coordinates'] \
    .str.split(',', expand=True).astype(float)
df[['org_latitude', 'org_longitude']] = df['Shipments: Locations Departure port Coordinates'] \
    .str.split(',', expand=True).astype(float)
df = df.drop(["Shipments: Locations Arrival port Coordinates","Shipments: Locations Departure port Coordinates"], axis=1)


In [4]:
cols_to_convert = [
    'Shipments: Attributes Gross Weight (kg)', 
    'Shipment Containers: Distance Final Port to Destination Distance in mile',
    'Shipment Containers: Distance Origin to First Port Distance in km'
]

# Remove commas and convert to float
df[cols_to_convert] = df[cols_to_convert].replace(',', '', regex=True).astype(float)


In [5]:
df.head()

,Shipments: Attributes Ocean type category,Shipments: Attributes Shipment ID,Shipments: Attributes Tradelane,Shipments: Attributes Gross Weight (kg),Shipments: Attributes Volume (cbm),Shipment Containers: Distance Final Port to Destination Distance in mile,Shipment Containers: Distance Origin to First Port Distance in km,Shipments: Attributes Freight Type,Port pair,Total Actual Days,dest_latitude,dest_longitude,org_latitude,org_longitude
0,coloaded_lcl,3051512,TAWB,323.0,2.622,160.13,358.66,Port to Door,DEHAM - USATL,70.0,32.776475,-79.931051,53.551085,9.993682
1,coloaded_lcl,3065451,TAWB,490.0,0.538,30.83,346.84,Door to Door,DEHAM - USCHI,45.0,45.501689,-73.567256,53.551085,9.993682
2,coloaded_lcl,3022941,TPEB,1913.0,11.360,54.37,233.94,Port to Door,BDCGP - USLAX,57.0,34.052234,-118.243685,22.356851,91.783182
3,coloaded_lcl,3024506,FEWB,752.0,0.508,147.02,421.03,Door to Door,INNSA - GBFXT,57.0,51.961726,1.351255,18.949936,72.951188
4,flxt_lcl,3055002,TPEB,6471.0,8.110,1793.56,296.68,Port to Door,TWKHH - USLAX,43.0,34.052234,-118.243685,22.627278,120.301435


In [6]:
# 1. Filter TPEB data
tpeb_df = df[df['Shipments: Attributes Tradelane'] == 'TPEB'].copy()

# 2. Feature Engineering
# Weight-to-volume ratio
tpeb_df['Weight_to_Volume'] = tpeb_df['Shipments: Attributes Gross Weight (kg)'] / (tpeb_df['Shipments: Attributes Volume (cbm)'] + 1e-6)

# Total shipment distance (km + converted miles)
tpeb_df['Total_Shipment_Distance_km'] = tpeb_df['Shipment Containers: Distance Origin to First Port Distance in km'] + (tpeb_df['Shipment Containers: Distance Final Port to Destination Distance in mile'] * 1.60934)

# Split port pair into origin and destination
tpeb_df[['Origin Port', 'Destination Port']] = tpeb_df['Port pair'].str.split(' - ', expand=True)


tpeb_df['Geo Distance'] = tpeb_df.apply(
    lambda row: geodesic(
        (row['org_latitude'], row['org_longitude']), 
        (row['dest_latitude'], row['dest_longitude'])
    ).km, axis=1
)

# # Label encode origin/destination ports
# le_origin = LabelEncoder()
# le_dest = LabelEncoder()
# tpeb_df['Origin Port'] = le_origin.fit_transform(tpeb_df['Origin Port'])
# tpeb_df['Destination Port'] = le_dest.fit_transform(tpeb_df['Destination Port'])

# Drop unused or now redundant columns
tpeb_df.drop([
    "Shipments: Attributes Tradelane", "Port pair",
    "Shipments: Attributes Gross Weight (kg)",
    "Shipments: Attributes Volume (cbm)"
], axis=1, inplace=True)



In [7]:
tpeb_df = tpeb_df.dropna()
tpeb_df.head()

,Shipments: Attributes Ocean type category,Shipments: Attributes Shipment ID,Shipment Containers: Distance Final Port to Destination Distance in mile,Shipment Containers: Distance Origin to First Port Distance in km,Shipments: Attributes Freight Type,Total Actual Days,dest_latitude,dest_longitude,org_latitude,org_longitude,Weight_to_Volume,Total_Shipment_Distance_km,Origin Port,Destination Port,Geo Distance
2,coloaded_lcl,3022941,54.37,233.94,Port to Door,57.0,34.052234,-118.243685,22.356851,91.783182,168.397873,321.439816,BDCGP,USLAX,13003.788261
4,flxt_lcl,3055002,1793.56,296.68,Port to Door,43.0,34.052234,-118.243685,22.627278,120.301435,797.903724,3183.127850,TWKHH,USLAX,11203.015045
9,coloaded_lcl,3021482,54.37,230.96,Port to Door,79.0,34.052234,-118.243685,22.356851,91.783182,185.529075,318.459816,BDCGP,USLAX,13003.788261
12,coloaded_lcl,3053937,1833.99,7.25,Port to Door,77.0,34.052234,-118.243685,10.823099,106.629664,151.164063,2958.763467,VNSGN,USLAX,13151.722926
14,flxt_lcl,2976454,9.93,45.30,Port to Door,37.0,34.052234,-118.243685,22.627278,120.301435,652.695684,61.280746,TWKHH,USLAX,11203.015045


In [8]:
from scipy.stats import zscore
df_clean = tpeb_df[(np.abs(zscore(tpeb_df.select_dtypes(include=[np.number]))) < 3).all(axis=1)]


In [9]:
df_clean.head()

,Shipments: Attributes Ocean type category,Shipments: Attributes Shipment ID,Shipment Containers: Distance Final Port to Destination Distance in mile,Shipment Containers: Distance Origin to First Port Distance in km,Shipments: Attributes Freight Type,Total Actual Days,dest_latitude,dest_longitude,org_latitude,org_longitude,Weight_to_Volume,Total_Shipment_Distance_km,Origin Port,Destination Port,Geo Distance
2,coloaded_lcl,3022941,54.37,233.94,Port to Door,57.0,34.052234,-118.243685,22.356851,91.783182,168.397873,321.439816,BDCGP,USLAX,13003.788261
4,flxt_lcl,3055002,1793.56,296.68,Port to Door,43.0,34.052234,-118.243685,22.627278,120.301435,797.903724,3183.127850,TWKHH,USLAX,11203.015045
9,coloaded_lcl,3021482,54.37,230.96,Port to Door,79.0,34.052234,-118.243685,22.356851,91.783182,185.529075,318.459816,BDCGP,USLAX,13003.788261
12,coloaded_lcl,3053937,1833.99,7.25,Port to Door,77.0,34.052234,-118.243685,10.823099,106.629664,151.164063,2958.763467,VNSGN,USLAX,13151.722926
14,flxt_lcl,2976454,9.93,45.30,Port to Door,37.0,34.052234,-118.243685,22.627278,120.301435,652.695684,61.280746,TWKHH,USLAX,11203.015045


In [10]:
# 3. Prepare X and y
X = df_clean.drop("Total Actual Days", axis=1)
y = df_clean["Total Actual Days"]

# One-hot encode any categorical (if still exists)
X = pd.get_dummies(X)

In [11]:
# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [12]:
# 5. Hyperparameter tuning
param_grid = {
    'n_estimators': [200, 500],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [5, 10],
    'max_depth': [None, 20]
}



In [13]:
grid = GridSearchCV(RandomForestRegressor(random_state=0), param_grid, cv=3, scoring='r2', n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [None, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [5, 10],
                         'n_estimators': [200, 500]},
             scoring='r2')

In [14]:
# 6. Evaluate best model
best_model = grid.best_estimator_
preds = best_model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("Best Parameters:", grid.best_params_)
print(f"Improved MAE: {mae:.2f}")
print(f"Improved R²: {r2:.4f}")

Best Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 500}
Improved MAE: 6.57
Improved R²: 0.6884
